<a href="https://colab.research.google.com/github/mdallas1/shared_code/blob/main/L8_spherical_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install octave

# Spherical Pendulum: a nontrivial problem

To end the semester, it feels right to solve a nontrivial problem where we don't know what the solution should necessarily look like, and we cannot (easily) solve the problem by hand. Section 8.10 in our book provides three examples of precisely this type of problem. We'll work through the first one.

The goal is to model the motion of a particle under the force of gravity constrained to move on a spherical surface with radius 1.

For you physics folks that have had theoretical mechanics, this problem will (probably) feel familiar. What I hope you take away from this problem is how we can pass from the analytical model to a numerical model, and then how we can solve this numerical model and verify that our solution is trustworthy despite not knowing the true solution.

For non-physics folks, my hope is that you see how a rather complicated problem can be abstracted to fit in the Cauchy problem framework we've studied this unit, and therefore we can apply all the same techniques. I also hope that you see how we can verify a solution is trustworthy when we do not know what the true solution should look like.

Plots will be very nice for this problem both for a sanity check and because they are pretty. Since I want to stick with Octave, I will include all the code below, but to see the plots you will need to copy the code into a local editor and run it there.


# How to reduce everything to a first order system

Ultimately, what we have in this problem is a particle subject to some forces. We can therefore apply Newton's second law, which tells us that if $x(t)$ is the position of a particle of mass $m$ at time $t\geq t_0$ subject to a net force $F(t)$, then

$$ x''(t) = \dfrac{F(t)}{m}.$$

We'll also refer to this as Newton's ODE. (This is just a term I decided to use here. It's not a standard term.) The methods we've studied for solving ODEs are designed for the Cauchy problem:

$$
\begin{align}
y'(t) &= f(t,y), \hspace{1em} t \text{ in } (t_0,T)\\
y(t_0) &= y_0
\end{align}
$$

Note that Newton's second law is a second order ODE, not a first order ODE. So to use our methods to solve Newton's ODE, we need to write it as a first order ODE. There is a standard way to do this, and it has a particularly nice interpration when applied to Newton's ODE.

Note that we can write $x''(t) = (x'(t))'$. If we now let $v(t) = x'(t)$, then $v'(t) = x''(t) = F(t)/m$. We thus obtain a **system** of two ODEs:

$$
\begin{align}
x'(t) &= v(t)\\
v'(t) &= \dfrac{F(t)}{m}.
\end{align}
$$

Interpreting $v(t)$ as the velocity of our particle at time $t$, what we have done is rewrite one equation describing the acceleration of the particle as two equations. One describes how the position changes, and the other describes how the velocity changes.

If we write this system of two ODEs in vector form by letting $\mathbf{y} = [x(t)\hspace{0.5em} v(t)]^T$ and $\mathbf{f}(t,\mathbf{y}) = [v(t)\hspace{0.5em} F(t)/m]^T$, we find that we now have a Cauchy problem!

$$
\begin{align}
\mathbf{y}'(t) &= \mathbf{f}(t,\mathbf{y})\\
\mathbf{y}(t_0) &= \mathbf{y}_0.
\end{align}
$$

To keep notation clean, you will typically see the components $x(t)$ and $v(t)$ of $\mathbf{y}(t)$ written as $y_1(t)$ and $y_2(t)$ respectively. The really nice thing about writing this second order equation as a system of first order equations is that once we have a Cauchy problem, we can apply forward Euler, backward Euler, or Crank-Nicolson without any significant alterations.

We will use precisely this trick in order to simulate the motion of a ball in a spherical bowl using these three methods. Our system, however, will not have two ODEs, but six. Since our ball can move in three dimensions, it's position $\mathbf{x}(t)$ has three components: $x_1(t)$, $x_2(t)$, and $x_3(t)$. You could also write these as $x(t)$, $y(t)$, and $z(t)$ respectively, but the subscript form is standard and easier to generalize.

Each of these components obeys Newton's second law. So we have three second order ODEs:

$$
\begin{align}
x_1''(t) &= F_1(t)/m \\
x_2''(t) &= F_2(t)/m \\
x_3''(t) &= F_3(t)/m \\
\end{align}
$$

We can still turn this into a Cauchy problem by applying our trick to each of these three second order equations. Thereby obtaining six first order equations of the form:

$$
\begin{align}
y_1'(t) &= y_4(t) \\
y_2'(t) &= y_5(t) \\
y_3'(t) &= y_6(t) \\
y_4'(t) &= F_1/m\\
y_5'(t) &= F_2/m\\
y_6'(t) &= F_3/m
\end{align}
$$

Note that $y_1(t) = x_1(t)$, $y_2(t)=x_2(t)$, and $y_3(t)=x_3(t)$. If this seems like a lot, note that we can still write the system very simply as

$$
\mathbf{y}' = \mathbf{f}(t,\mathbf{y}).
$$

So once we define $\mathbf{f}$ in the code, we simply run the methods like we did with a single equation.

<details>
<summary><b>More detailed description of problem setup. </b></summary>

# Setup with additional details

Let's set up the differential equation we want to solve. We'll use Newton's second law, which says that the position $\mathbf{x}(t)$ at time $t\geq 0$ of a particle of mass $m$ subject to a force field $\mathbf{F}(t)$ satisfies the second order linear *system* of ODEs

$$ \mathbf{x}''(t) = \dfrac{\mathbf{F}(t)}{m}.$$

Notice that this is a system of three second order differential equations with three unknown functions: $x_1(t)$, $x_2(t)$, and $x_3(t)$. Written out fully, and noting that $\mathbf{F}(t) = (F_1(t),F_2(t),F_3(t))^T$, we have

$$
\begin{align}
x_1''(t) &= F_1(t)/m \\
x_2''(t) &= F_2(t)/m \\
x_3''(t) &= F_3(t)/m \\
\end{align}
$$

Any degree $d$ ODE can be transformed into a system of $d$ first order equations. It follows that a system of $n$ second order equations can always be transformed into a system of $2n$ first order equations. This trick will allow us to transform this problem into a Cauchy problem. Let $\Phi(\mathbf{x})$ denote the constraint equation:

$$\Phi(\mathbf{x}(t)) = x_1^2+x_2^2+x_3^2-1.$$

Since our particle is constrained to move on a unit sphere, we require that

$$\Phi(\mathbf{x}(t)) = 0$$

for all $t\geq 0$. This function will come into play as one of the forces acting on our particle.

To follow our book's notation, for $i=1,2,3$, let $y_i = x_i$, and $y_{i+3} = x_i'$. We thus obtain the first order system

$$
\begin{align}
y_i'(t) &= y_{i+3}, \text{ and } \\
y_{i+3}'(t) &= \dfrac{1}{m}\left(F_i - \lambda\dfrac{\partial \Phi}{\partial y_i}\right).
\end{align}
$$

This is a system of six first order ODEs with six unknown functions $y_i$, $i=1,2,...6$.  Note that $F_1 = F_2 = 0$, $F_3 = -gm$, where $g = 9.8 m/s^2$ is magnitude of the acceleration due to gravity near Earth's surface, and $\partial \Phi/\partial y_i = 2y_i$. Hence we can explicitly express the whole system as

$$
\begin{align}
y_1'(t) &= y_4(t) \\
y_2'(t) &= y_5(t) \\
y_3'(t) &= y_6(t) \\
y_4'(t) &= -\dfrac{2\lambda}{m}y_1(t) \\
y_5'(t) &= -\dfrac{2\lambda}{m}y_2(t) \\
y_6'(t) &= -\dfrac{1}{m}\left( gm + 2\lambda y_3(t)\right)
\end{align}
$$

or in matrix-vector form:

$$
\begin{bmatrix} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \\ y_6\end{bmatrix}'
 = \begin{bmatrix} 0 &0 &0 &1 &0 &0 \\
 0 &0 &0 &0 &1 &0 \\ 0 &0 &0 &0 &0 &1 \\
 -2\lambda/m &0 &0 &0 &0 &0 \\
 0 &-2\lambda/m &0 &0 &0 &0 \\
 0 &0 &-2\lambda/m  &0 &0 &0
 \end{bmatrix}\begin{bmatrix} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \\ y_6\end{bmatrix} + \begin{bmatrix} 0 \\0 \\0 \\0 \\0 \\-g\end{bmatrix}
$$

We'll let $\mathbf{y}(t) = (y_1,y_2,y_3,...,y_6)^T$. Here $\lambda$ is a constant, but not a particularly pretty one. See page 314 for an explicit form of $\lambda$, and see [here](https://people.computing.clemson.edu/~jtessen/cpsc6190/html/lectures/constrained/sphere_constrained_slides.pdf) for a derivation of the equations of motion. Since we will know the initial position and initial velocity of the mass, we have an initial conditoin $\mathbf{y}(0) = \mathbf{y}_0$. We now have a nice *linear* Cauchy problem:

$$
\begin{align}
\mathbf{y}'(t) &= A\mathbf{y}+\mathbf{b}\\
\mathbf{y}(0) &= \mathbf{y}_0
\end{align}
$$


The fact that these are vector quantities makes little difference in our implementation. We can still apply forward Euler, backward Euler, and Crank-Nicolson. That's what we'll do next.
</details>

# Solvers and RHS function

In [ ]:
#@title Forward Euler
%%writefile fwd_euler.m

function y = fwd_euler(t0,T,y0,f,N)
  % ==============================
  % FORWARD EULER METHOD SOLVING
  % CAUCHY PROBLEM y' = f(t,y)
  % ON [t0,T] with y(0) = y0.
  % N IS THE NUMBER OF NODES.
  % ==============================
  y = zeros(6,N); y(:,1)=y0; h = (T-t0)/(N-1);
  nodes = t0:h:T;
  for k = 2:N
    yprev = y(:,k-1);
    y(:,k) = yprev + h * f(nodes(k-1),yprev);
  end

In [ ]:
#@title Backward Euler
%%writefile bck_euler.m

function y = bck_euler(t0,T,y0,f,N)
  % ==============================
  % BACKWARD EULER METHOD SOLVING
  % CAUCHY PROBLEM y' = f(t,y)
  % ON [t0,T] with y(0) = y0.
  % N IS THE NUMBER OF NODES.
  % ==============================
  y = zeros(6,N); y(:,1)=y0; h = (T-t0)/(N-1);
  nodes = t0:h:T;
  for k = 2:N
    yprev = y(:,k-1);
    euler_fun = @(u) u - yprev - h * f(nodes(k),u);
    y(:,k) = fsolve(euler_fun,yprev);
  end

In [ ]:
#@title Crank-Nicolson
%%writefile crank_nicolson.m
function y = crank_nicolson(t0,T,y0,f,N)
  % ==============================
  % CRANK-NICOLSON METHOD SOLVING
  % CAUCHY PROBLEM y' = f(t,y)
  % ON [t0,T] with y(0) = y0.
  % N IS THE NUMBER OF NODES.
  % ==============================
  y = zeros(6,1); y(:,1)=y0; h = (T-t0)/(N-1);
  nodes = t0:h:T;
  for k = 2:N
    yprev = y(:,k-1);
    euler_fun = @(u) u - yprev - 0.5*h*( f(nodes(k),u)+f(nodes(k-1),yprev) );
    y(:,k) = fsolve(euler_fun,yprev);
  end

In [ ]:
#@title Spherical Pendulum RHS
%%writefile frhs.m
function out = frhs(t,y)
  mass = 1; g = 9.8;
  Phix = 2*y(1); Phiy = 2*y(2); Phiz = 2*y(3);
  G = [Phix ; Phiy ; Phiz];
  F = [0 ; 0 ; -g*mass];

  b = zeros(6,1); b(end) = -g;
  lambda = (2*mass * dot(y(4:6),y(4:6)) + dot(G,F)) / dot(G,G);
  A = zeros(6,6); u = -2*lambda/mass * ones(3,1);
  A = A + diag(ones(3,1),3) + diag(u,-3);

  out = A*y + b;
end

# Solve

In [ ]:
%%writefile solve_pendulum.m

% COPY THIS FILE INTO LOCAL EDITOR TO SEE PLOTS

close all
t0 = 0; T=25; y0=[0 1 0 0.8 0 1.2]';
ff = @(t,y) frhs(t,y);
y_fe = fwd_euler(t0,T,y0,ff,10e3);
y_be = bck_euler(t0,T,y0,ff,20e3);
y_cn = crank_nicolson(t0,T,y0,ff,1e3);
tt = t0:(T-t0)/(1e4-1):T;

r_fe = max(abs(y_fe(1,:).^2 + y_fe(2,:).^2 + y_fe(3,:).^2 - 1))
r_be = max(abs(y_be(1,:).^2 + y_be(2,:).^2 + y_be(3,:).^2 - 1))
r_cn = max(abs(y_cn(1,:).^2 + y_cn(2,:).^2 + y_cn(3,:).^2 - 1))

%figure(1), hold on, grid on
%plot3(y_fe(1,:),y_fe(2,:),y_fe(3,:))
%plot3(y0(1),y0(2),y0(3),'or','Linewidth',1.2)
%title("Forward Euler Solution","FontSize",16);

%figure(2), hold on, grid on
%plot3(y_be(1,:),y_be(2,:),y_be(3,:))
%plot3(y0(1),y0(2),y0(3),'or','Linewidth',1.2)
%title("Backward Euler Solution","FontSize",16);

%figure(3), hold on, grid on
%plot3(y_cn(1,:),y_cn(2,:),y_cn(3,:))
%plot3(y0(1),y0(2),y0(3),'or','Linewidth',1.2)
%title("Crank-Nicolson Solution","FontSize",16);


In [ ]:
!octave -W solve_pendulum.m

# How do we know which solution to trust?

Upon plotting the solutions obtained from each method, we get three figures that all look sort of right, with perhaps one looking slightly better. The eyeball test has its uses, but if we we required a certain tolerance to be met in our simulation, which is usually the case, we'd need a more precise measure of the error. We don't, however, have a the true solution.

In these cases we rely on an **error estimator**, but what's an appropriate estimator in this problem? One thing we can check is the constraint. If our solution is accurate, it should obey the constraint

$$y_1(t)^2+y_2(t)^2 + y_3^2(t) = 1.$$

We know $(y_1(t),y_2(t),y_3(t))^T$ at the nodes $t_n$, so we'll compute $r(\mathbf{y}):=y_1(t)^2+y_2(t)^2 + y_3^2(t)  - 1$ at each node, and take the max value to be the error estimator.

 If you add a line to solve\_pendulum.m to compute this, you find that $r(\mathbf{y}_{fe}) = 1.0579$, $r(\mathbf{y}_{be}) = 0.5806$, and $r(\mathbf{y}_{cn}) = 0.095830$. Observing the plots (you'll to copy the code locally to plot) the Crank-Nicolson method does seem to give the best solution, and this is confirmed using the contstraint as our error estimator.

# Solve using built in high order methods

To conclude our discussion of the spherical pendulum and numerical solutions to ODEs, we'll demonstrate the use of higher order methods that are implemented efficiently with built in MATLAB and Octave functions (and python as well; see below).

Higher order methods are often built from **multistep methods**. Both forward Euler and backward Euler are *one step* methods: they use information about the derivative at a single point to inform the update at each step. Crank-Nicolson is a multistep method, specifically a 2-step method, since it uses information about the derivative at two places to inform the udpate step.

Crank-Nicolson is more accurate than forward and backward Euler, which suggests that using more than one step yields a more accurate method. This is indeed the case, and motivates the use of even more steps. We won't discuss these explicitly here, but you can read sections 8.7 and 8.8 in our book for explicit examples. Here our focus is on calling them from built in functions.

The examples below, ode45 and ode23, are two standard solvers used by mathematicians and scientists everyday. For information on other solvers available in MATLAB and Octave, and how to know how to choose a solver, check out [this page](https://www.mathworks.com/help/matlab/math/choose-an-ode-solver.html).

For Python users, you can read about the available solvers in the scipy.integrate library [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).


## ODE45

The MATLAB/Octave function ode45 implements a fourth order multistep method. That is, the method converges like $\mathcal{O}(h^4)$. This is even better than Crank-Nicolson's $\mathcal{O}(h^2)$ convergence. It can be used to solve any Cauchy problem. To call ode45, you give it three arguments: the function $\mathbf{f}$ on the right-hand-side, the time interval, and the initial condition. It returns the nodes $t_n$ and the solution $y$. Let's see this in action.


In [ ]:
#@title Solve Pendulum with ODE45
%%writefile solve_pendulum_ode45.m

t0 = 0; T=25; y0=[0 1 0 0.8 0 1.2]';
ff = @(t,y) frhs(t,y);

[nodes,y_ode45] = ode45(ff,[0 T],y0,odeset('RelTol',1e-4));

% TAKE TRANSPOSE TO MATCH OUR FORMAT
y_ode45 = y_ode45' ;

% CHECK CONSTRAINT
r_ode45 = max(abs(y_ode45(1,:).^2 + y_ode45(2,:).^2 + y_ode45(3,:).^2 - 1))

% COPY INTO LOCAL EDITOR TO SEE PLOTS
figure(4), hold on, grid on
plot3(y_ode45(1,:),y_ode45(2,:),y_ode45(3,:))
plot3(y0(1),y0(2),y0(3),'or','Linewidth',1.2)
title("ODE45 Solution","FontSize",16);

function out = frhs(t,y)
  mass = 1; g = 9.8;
  Phix = 2*y(1); Phiy = 2*y(2); Phiz = 2*y(3);
  G = [Phix ; Phiy ; Phiz];
  F = [0 ; 0 ; -g*mass];

  b = zeros(6,1); b(end) = -g;
  lambda = (2*mass * dot(y(4:6),y(4:6)) + dot(G,F)) / dot(G,G);
  A = zeros(6,6); u = -2*lambda/mass * ones(3,1);
  A = A + diag(ones(3,1),3) + diag(u,-3);

  out = A*y + b;
end

In [ ]:
!octave -W solve_pendulum_ode45.m

## ODE23

ode23 is called the same way as ode45, and implements an order 3 method. Let's see how it performs on the pendulum problem.

In [ ]:
#@title Solve Pendulum with ODE23
%%writefile solve_pendulum_ode23.m

t0 = 0; T=25; y0=[0 1 0 0.8 0 1.2]';
ff = @(t,y) frhs(t,y);

[nodes,y_ode23] = ode23(ff,[0 T],y0,odeset('RelTol',1e-4));

% TAKE TRANSPOSE TO MATCH OUR FORMAT
y_ode23 = y_ode23' ;

% CHECK CONSTRAINT
r_ode23 = max(abs(y_ode23(1,:).^2 + y_ode23(2,:).^2 + y_ode23(3,:).^2 - 1))

% COPY INTO LOCAL EDITOR TO SEE PLOTS
figure(5), hold on, grid on
plot3(y_ode23(1,:),y_ode23(2,:),y_ode23(3,:))
plot3(y0(1),y0(2),y0(3),'or','Linewidth',1.2)
title("ODE23 Solution","FontSize",16);

function out = frhs(t,y)
  mass = 1; g = 9.8;
  Phix = 2*y(1); Phiy = 2*y(2); Phiz = 2*y(3);
  G = [Phix ; Phiy ; Phiz];
  F = [0 ; 0 ; -g*mass];

  b = zeros(6,1); b(end) = -g;
  lambda = (2*mass * dot(y(4:6),y(4:6)) + dot(G,F)) / dot(G,G);
  A = zeros(6,6); u = -2*lambda/mass * ones(3,1);
  A = A + diag(ones(3,1),3) + diag(u,-3);

  out = A*y + b;
end

In [ ]:
!octave -W solve_pendulum_ode23.m